In [ ]:
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
%env LLM_API_KEY=替换为自己的Qwen API Key
%env ZHIPU_API_KEY=替换为自己的智谱API Key，如果不需要测试智谱可以不输入并跳过相应代码

In [2]:
%%capture --no-stderr
!pip install -U langchain langchain_community langchain_openai pypdf sentence_transformers chromadb shutil openpyxl FlagEmbedding

In [3]:
import os
import pandas as pd

In [4]:
expr_version = 'generation_v1_long_context_no_rag'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

os.makedirs(expr_dir, exist_ok=True)

# 读取文档

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

loader = PyPDFLoader(os.path.join(os.path.pardir, 'data', '2024全球经济金融展望报告.pdf'))
documents = loader.load()

In [6]:
import re

pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024年"

In [7]:
docs = [Document(page_content='\n'.join(re.sub(pattern, '', doc.page_content) for doc in documents))]

In [8]:
qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))

# 生成答案

In [9]:
from langchain.llms import Ollama

qwen2_7b_llm = Ollama(
    model='qwen2:7b-instruct',
    base_url="http://localhost:11434"
)
qwen2_7b_32k_llm = Ollama(
    model='qwen2:7b-instruct-32k',
    base_url="http://localhost:11434"
)

In [10]:
from langchain_openai import ChatOpenAI

glm4_plus_llm = ChatOpenAI(
    model='glm-4-plus',
    base_url='https://open.bigmodel.cn/api/paas/v4/',
    api_key=os.environ['ZHIPU_API_KEY']
)

In [11]:
prompt_tmpl = """
你是一个金融分析师，擅长根据所获取的信息片段，对问题进行分析和推理。
你的任务是根据所获取的信息片段（<<<<context>>><<<</context>>>之间的内容）回答问题。
回答保持简洁，不必重复问题，不要添加描述性解释和与答案无关的任何内容。
已知信息：
<<<<context>>>
{{knowledge}}
<<<</context>>>
问题：{{query}}
请回答：
""".replace('{{knowledge}}', '\n\n'.join([doc.page_content for doc in docs])).strip()

In [12]:
len(prompt_tmpl)

31333

In [12]:
def get_answer(llm, query):
    prompt = prompt_tmpl.replace('{{query}}', query)

    resp = llm.invoke(prompt)
    return resp if isinstance(resp, str) else resp.content

In [13]:
get_answer(qwen2_7b_llm, '报告的发布机构是什么？')

'报告的发布机构是中国银行研究院。'

In [14]:
get_answer(qwen2_7b_32k_llm, '报告的发布机构是什么？')

'报告的发布机构是中国银行研究院。'

In [15]:
get_answer(glm4_plus_llm, '报告的发布机构是什么？')

'中国银行研究院'

## 预测

In [16]:
from tqdm.auto import tqdm

prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

def predict(llm, prediction_df):
    prediction_df = prediction_df.copy()
    answer_dict = {}

    for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        uuid = row['uuid']
        question = row['question']
        answer = get_answer(llm, question)
        answer_dict[question] = {
            'uuid': uuid,
            'ref_answer': row['ref_answer'],
            'gen_answer': answer
        }
        
    prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])
    return prediction_df

In [17]:
pred_dfs = {}

In [18]:
pred_dfs['ollama_qwen2_7b'] = predict(qwen2_7b_llm, prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
pred_dfs['ollama_qwen2_7b_32k'] = predict(qwen2_7b_32k_llm, prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

**特别注意：**glm4 plus比较贵，百万token需要50元，使用长上下文批量预测，如果不是赠送的token就不要尝试了，一定要观察自己的剩余token，余额为负了不会实时提醒，到时候可能会欠很多钱

In [22]:
pred_dfs['glm4_plus'] = predict(glm4_plus_llm, prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
pred_dfs['ollama_qwen2_7b'].head()

,uuid,question,qa_type,ref_answer,gen_answer
0,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布机构是什么？,detailed,中国银行研究院,报告的发布机构是中国银行研究院。
1,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布日期是什么时候？,detailed,2023年12月12日,报告中未直接提供具体的发布日期。通常，这样的信息会在报告的封面上或前言部分明确标注。由于没有...
2,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,2023年全球经济增长有什么特点？,detailed,全球经济增长动力持续回落，各国复苏分化，发达经济体增速放缓，新兴经济体表现稳定。,由于提供的文本内容与2023年全球经济增长的特点无关，我将提供一个通用的经济趋势分析。\n\...
3,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,全球贸易增长情况如何？,detailed,全球贸易增长乏力。,关于全球贸易增长的具体数据和趋势分析需要查阅最新的经济报告或者统计数据来获取最准确的信息。通...
5,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,展望2024年，全球经济复苏的预期如何？,detailed,全球经济复苏预计将依旧疲软。,文章中未直接提供对2024年全球经济复苏的具体预测或预期。文中主要分析了美国房地产市场面临的...


In [25]:
pred_dfs['ollama_qwen2_7b_32k'].head()

,uuid,question,qa_type,ref_answer,gen_answer
0,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布机构是什么？,detailed,中国银行研究院,报告的发布机构是中国银行研究院。
1,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布日期是什么时候？,detailed,2023年12月12日,报告的发布日期是2023年12月12日。
2,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,2023年全球经济增长有什么特点？,detailed,全球经济增长动力持续回落，各国复苏分化，发达经济体增速放缓，新兴经济体表现稳定。,根据报告，2023年全球经济增长动力持续回落，各国复苏分化：\n\n1. 各国经济增长存在较...
3,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,全球贸易增长情况如何？,detailed,全球贸易增长乏力。,根据文中信息，全球贸易增长乏力。具体来说：\n\n在全球供应链持续恢复的同时，生产景气度逐渐...
5,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,展望2024年，全球经济复苏的预期如何？,detailed,全球经济复苏预计将依旧疲软。,根据文中内容，对于2024年全球经济增长的主要预期是：\n\n“复苏+分化”将成为2024年...


In [26]:
pred_dfs['glm4_plus'].head()

,uuid,question,qa_type,ref_answer,gen_answer
0,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布机构是什么？,detailed,中国银行研究院,中国银行研究院
1,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,报告的发布日期是什么时候？,detailed,2023年12月12日,2023年12月12日
2,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,2023年全球经济增长有什么特点？,detailed,全球经济增长动力持续回落，各国复苏分化，发达经济体增速放缓，新兴经济体表现稳定。,2023年全球经济增长动力持续回落，经济增速连续两年下降。
3,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,全球贸易增长情况如何？,detailed,全球贸易增长乏力。,全球贸易增长乏力，主要经济体出口贸易同比增速下降。
5,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,展望2024年，全球经济复苏的预期如何？,detailed,全球经济复苏预计将依旧疲软。,预计2024年全球经济复苏将依旧疲软，主要经济体增长态势和货币政策走势将进一步分化。


# 评估

In [27]:
import time
from langchain_openai import ChatOpenAI

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0.0001
)

def evaluate(judge_llm, prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [28]:
for key, pred_df in pred_dfs.items():
    pred_df['raw_score'] = evaluate(judge_llm, pred_df)
    pred_df['score'] = (pred_df['raw_score'] == '是').astype(int)
    print(f"{key} unique raw score: {pred_df['raw_score'].unique()}")

  0%|          | 0/100 [00:00<?, ?it/s]

ollama_qwen2_7b unique raw score: ['是' '否']


  0%|          | 0/100 [00:00<?, ?it/s]

ollama_qwen2_7b_32k unique raw score: ['是' '否']


  0%|          | 0/100 [00:00<?, ?it/s]

glm4_plus unique raw score: ['是' '否']


In [30]:
from IPython import display

In [32]:
for key, pred_df in pred_dfs.items():
    pred_df['score'] = (pred_df['raw_score'] == '是').astype(int)
    print(f"{key} accuracy: {pred_df['score'].mean()}")

    display.display_html(pred_df[['qa_type', 'score']].groupby('qa_type').mean().reset_index())

ollama_qwen2_7b accuracy: 0.06


,qa_type,score
0,detailed,0.064516
1,large_context,0.000000


ollama_qwen2_7b_32k accuracy: 0.68


,qa_type,score
0,detailed,0.666667
1,large_context,0.857143


glm4_plus accuracy: 0.86


,qa_type,score
0,detailed,0.849462
1,large_context,1.000000


In [33]:
for key, pred_df in pred_dfs.items():
    pred_df.to_excel(os.path.join(expr_dir, f'{expr_version}_{key}_prediction.xlsx'), index=False)